#### **Lip Reading Application**

##### Install/Import dependencies

In [ ]:
%%capture
#Install dependencies and confirm installation with !pip list
!pip install opencv-python matplotlib imageio gdown tensorflow-macos
!pip list

In [ ]:
#Import dependencies
import os
import cv2
#import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import gdown
from tensorflow import keras

#Neural Network dependencies
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

##### Load Data

In [ ]:
# %%capture
# #Downloading dataset that was made for creating lip-read models 
# UNCOMMENT FOR FIRST RUN
# url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
# output = 'data.zip'
# gdown.download(url, output, quiet = False)
# gdown.extractall('data.zip')

In [ ]:
def load_video(path) -> List[float]:
    '''
    Takes in a path to a video and returns the float values for each frame
        Args: 
            path: str --> path to video that will be passed into model
        Returns:
            List of floats that represents video1
    '''
    cap = cv2.VideoCapture(path)
    frames=[]
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        success, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        #isolates the mouth --> we can also use a lip detector to isolate the mouth
        frames.append(frame[190:236,80:220,:])
    cap.release()
    
    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames-mean), tf.float32)/std                 
    

In [ ]:
vocab=[x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

In [ ]:
char_num = tf.keras.layers.StringLookup(vocabulary = vocab, oov_token="")
num_char = tf.keras.layers.StringLookup(vocabulary = char_num.get_vocabulary(), oov_token="", invert = True)
print(f'The vocab is: {char_num.get_vocabulary()}'
      f'(size={char_num.vocabulary_size()})')

The vocab is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' '](size=40)


In [ ]:
def load_alignments(path) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2]!='sil':
            tokens=[*tokens,' ', line[2]]
    return char_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [ ]:
def load_data(path):
    path = bytes.decode(path.numpy())
    file_name = path.split('/')[-1].split('.')[0]
    video_path = os.path.join('data','s1',f'{file_name}.mpg')
    alignment_path = os.path.join('data','alignments', 's1',f'{file_name}.align')
    frames = load_video(video_path)
    alignments = load_alignments(alignment_path)
    return frames, alignments

In [ ]:
def mappable_function(path) -> List[str]:
    return tf.py_function(load_data,[path], (tf.float32,tf.int64))

##### Tensorflow Data Pipeline

In [ ]:
data = tf.data.Dataset.list_files('./data/s1/*.mpg')
data = data.shuffle(500)
data = data.map(mappable_function)
#batching into groups of 2, padding 75 frames with 40 tokens
data = data.padded_batch(2, padded_shapes=([75,None,None,None],[40]))
data = data.prefetch(tf.data.AUTOTUNE)

In [ ]:
#the data is now in batches of 2 alignments and 2 frames
frames, alignments = data.as_numpy_iterator().next()
alignments, len(frames)

2023-07-18 15:33:25.281351: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1000]
	 [[{{node Placeholder/_0}}]]
2023-07-18 15:33:25.282402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1000]
	 [[{{node Placeholder/_0}}]]


(array([[19,  5, 20, 39, 18,  5,  4, 39,  9, 14, 39, 14, 39, 19,  5, 22,
          5, 14, 39,  1,  7,  1,  9, 14,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  9, 14, 39,  7, 18,  5,  5, 14, 39,  9, 14, 39, 14, 39, 20,
         23, 15, 39, 16, 12,  5,  1, 19,  5,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0]]),
 2)

In [ ]:
sample = data.as_numpy_iterator()
val=sample.next()

##### Tensorflow Neural Network

In [ ]:

model = Sequential()
model.add(Conv3D(128,3,input_shape=(75,46,140,1),padding='same',activation='relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(256,3,padding='same',activation='relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(75,3,padding='same',activation='relu'))
model.add(MaxPool3D((1,2,2)))

model.add(TimeDistributed(Flatten()))
model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))
model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Dense(char_num.vocabulary_size()+1, kernel_initializer='he_normal', activation='softmax'))

2023-07-18 15:36:56.885296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-18 15:36:56.885994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-18 15:36:56.886799: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_3 (Conv3D)           (None, 75, 46, 140, 128)  3584      
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 75, 23, 70, 128)  0         
 3D)                                                             
                                                                 
 conv3d_4 (Conv3D)           (None, 75, 23, 70, 256)   884992    
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 75, 11, 35, 256)  0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 75, 11, 35, 75)    518475    
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 75, 5, 17, 75)   

In [ ]:
#check what our model returns
yhat = model.predict(val[0])

2023-07-18 15:37:04.619569: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-18 15:37:04.620290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-18 15:37:04.620916: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 4s 4s/step


In [ ]:
#Prediction by the model before training
tf.strings.reduce_join([num_char(tf.argmax(x)) for x in yhat[0]])

<tf.Tensor: shape=(), dtype=string, numpy=b"iii''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''!'''">

##### Training the Model

In [ ]:
def learning_rate(epoch, lr):
    if epoch<30:
        return lr
    else:
        return lr*tf.math.exp(-.1)

In [ ]:
#Loss function from keras Automatic Speech Recognition using CTC
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype='int64')
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:
class ProduceExample(tf.keras.callbacks.Callback): 
    def __init__(self, dataset) -> None: 
        self.dataset = dataset.as_numpy_iterator()
    
    def on_epoch_end(self, epoch, logs=None) -> None:
        data = self.dataset.next()
        yhat = self.model.predict(data[0])
        decoded = tf.keras.backend.ctc_decode(yhat, [75,75], greedy=False)[0][0].numpy()
        for x in range(len(yhat)):           
            print('Original:', tf.strings.reduce_join(num_char(data[1][x])).numpy().decode('utf-8'))
            print('Prediction:', tf.strings.reduce_join(num_char(decoded[x])).numpy().decode('utf-8'))
            print('~'*100)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=CTCLoss)

In [ ]:
checkpoint_callback = ModelCheckpoint(os.path.join('models','checkpoint'), monitor='loss', save_weights_only=True)
schedule_callback = LearningRateScheduler(learning_rate)
example_callback = ProduceExample(data)

In [ ]:
model.fit(data, epochs=3, callbacks=[checkpoint_callback, schedule_callback, example_callback])

Epoch 1/3


2023-07-18 15:50:28.231684: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-18 15:50:28.232412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-18 15:50:28.233383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

  2/500 [..............................] - ETA: 3:53:40 - loss: 208.8805

[mpeg1video @ 0x7f9ffd64a6c0] ac-tex damaged at 22 17
[mpeg1video @ 0x7f9ffd64a6c0] Warning MVs not available


  4/500 [..............................] - ETA: 3:48:00 - loss: 182.7903